# Introducing IMDB Ratings

> [Yalim Demirkesen](github.com/demirkeseny)



In [1]:
# necessary libraries:
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.corpus import wordnet
from ast import literal_eval

In [4]:
# uploading the data
movies = pd.read_csv('C:/Users/demir/Desktop/movie-recommender/data/movies_metadata.csv')

C:\Users\demir\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3018: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
movies.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [6]:
movies = movies[movies['status'] == 'Released'] 

In [7]:
# since I don't need most of the columns, I drop them using drop()
movies.drop(['homepage','original_language','belongs_to_collection',
             'overview','spoken_languages','tagline','video', 'status',
             'poster_path','imdb_id','adult','runtime','runtime',
             'revenue','release_date','production_countries','production_companies',
             'budget'], 
             axis=1, inplace=True)

In [8]:
movies.head()

,genres,id,original_title,popularity,title,vote_average,vote_count
0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,Toy Story,21.9469,Toy Story,7.7,5415.0
1,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,Jumanji,17.0155,Jumanji,6.9,2413.0
2,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,Grumpier Old Men,11.7129,Grumpier Old Men,6.5,92.0
3,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,Waiting to Exhale,3.85949,Waiting to Exhale,6.1,34.0
4,"[{'id': 35, 'name': 'Comedy'}]",11862,Father of the Bride Part II,8.38752,Father of the Bride Part II,5.7,173.0


In [9]:
movies.isnull().sum()

genres            0
id                0
original_title    0
popularity        0
title             0
vote_average      0
vote_count        0
dtype: int64

In [10]:
movies.dtypes

genres             object
id                 object
original_title     object
popularity         object
title              object
vote_average      float64
vote_count        float64
dtype: object

In [11]:
# since it may not stored as integers, I need to convert strings into integers
movies['popularity'] = pd.to_numeric(movies['popularity'])

In [12]:
movies.shape

(45014, 7)

In [13]:
# creating a list of all the genres stored in the dataframe
listt = []
some_lst=[]
for i in range(len(literal_eval(movies['genres'][0]))):
    some_lst.append(literal_eval(movies['genres'][0])[i]['name'])
listt.append(some_lst)
listt

[['Animation', 'Comedy', 'Family']]

In [14]:
# creating a dataframe including the genres of each movie
# since there might be multiple some additional columns will be created
df=pd.DataFrame(None)
for j in range(len(movies)):
           
    some_lst=[]
    same_lst=[]
    try:
        for i in range(len(literal_eval(movies['genres'][j]))):
            some_lst.append(literal_eval(movies['genres'][j])[i]['name'])
    except:
        pass
    same_lst.append(some_lst)
    df1 = pd.DataFrame(same_lst)
    df = pd.concat([df,df1])

In [15]:
df = df.reset_index(drop=True)

In [16]:
# concatination of the data frame created with the movie data frame
movies = pd.concat([movies, df], axis=1)

In [17]:
movies.drop(['genres'], axis=1, inplace=True)

In [18]:
movies.head()

,id,original_title,popularity,title,vote_average,vote_count,0,1,2,3,4,5,6,7
0,862,Toy Story,21.946943,Toy Story,7.7,5415.0,Animation,Comedy,Family,NaN,NaN,NaN,NaN,NaN
1,8844,Jumanji,17.015539,Jumanji,6.9,2413.0,Adventure,Fantasy,Family,NaN,NaN,NaN,NaN,NaN
2,15602,Grumpier Old Men,11.712900,Grumpier Old Men,6.5,92.0,Romance,Comedy,NaN,NaN,NaN,NaN,NaN,NaN
3,31357,Waiting to Exhale,3.859495,Waiting to Exhale,6.1,34.0,Comedy,Drama,Romance,NaN,NaN,NaN,NaN,NaN
4,11862,Father of the Bride Part II,8.387519,Father of the Bride Part II,5.7,173.0,Comedy,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Below is the rating system used by IMDB. This is very important method since all the movies that are rated 5 out of 5 are not the same. One might have got millions of ratings whereas one might be more limited in terms of number of votes. So this rating gives more importance to the movies that are rated more.

$$ (WR)=(v/(v+m))R+(m/(v+m))C $$

- R = average for the movie (mean)
- v = number of votes for the movie
- m = minimum votes required to be listed in the Top 50 (currently 1000)
- C = the mean vote across the whole report (currently 6.8)

In [19]:
C = movies['vote_average'].mean()

In [20]:
m = movies['vote_count'].quantile(0.95)

In [21]:
v = movies['vote_count']

In [22]:
R = movies['vote_average']

In [24]:
# we will get only the 0.95 quantile of the movies to make the analysis more smoother:
filtered = movies[movies['vote_count'] >= m]

In [25]:
filtered.head()

,id,original_title,popularity,title,vote_average,vote_count,0,1,2,3,4,5,6,7
0,862,Toy Story,21.946943,Toy Story,7.7,5415.0,Animation,Comedy,Family,NaN,NaN,NaN,NaN,NaN
1,8844,Jumanji,17.015539,Jumanji,6.9,2413.0,Adventure,Fantasy,Family,NaN,NaN,NaN,NaN,NaN
5,949,Heat,17.924927,Heat,7.7,1886.0,Action,Crime,Drama,Thriller,NaN,NaN,NaN,NaN
9,710,GoldenEye,14.686036,GoldenEye,6.6,1194.0,Adventure,Action,Thriller,NaN,NaN,NaN,NaN,NaN
15,524,Casino,10.137389,Casino,7.8,1343.0,Drama,Crime,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
# created another column to indicate this evaluation
filtered['wr'] =  (v/(v+m) * R) + (m/(m+v) * C)

C:\Users\demir\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [27]:
# then this column is used to sort
filtered.sort_values(by=['wr'], inplace= True, ascending=False)

C:\Users\demir\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [28]:
filtered

,id,original_title,popularity,title,vote_average,vote_count,0,1,2,3,4,5,6,7,wr
314,278,The Shawshank Redemption,51.645403,The Shawshank Redemption,8.5,8358.0,Drama,Crime,NaN,NaN,NaN,NaN,NaN,NaN,8.356664
834,238,The Godfather,41.109264,The Godfather,8.5,6024.0,Drama,Crime,NaN,NaN,NaN,NaN,NaN,NaN,8.304895
12481,155,The Dark Knight,123.167259,The Dark Knight,8.3,12269.0,Drama,Action,Crime,Thriller,NaN,NaN,NaN,NaN,8.207678
2843,550,Fight Club,63.869599,Fight Club,8.3,9678.0,Drama,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.184033
292,680,Pulp Fiction,140.950236,Pulp Fiction,8.3,8670.0,Thriller,Crime,NaN,NaN,NaN,NaN,NaN,NaN,8.171199
351,13,Forrest Gump,48.307194,Forrest Gump,8.2,8147.0,Comedy,Drama,Romance,NaN,NaN,NaN,NaN,NaN,8.068459
522,424,Schindler's List,41.725123,Schindler's List,8.3,4436.0,Drama,History,War,NaN,NaN,NaN,NaN,NaN,8.059319
23673,244786,Whiplash,64.299990,Whiplash,8.3,4376.0,Drama,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.056319
5481,129,千と千尋の神隠し,41.048867,Spirited Away,8.3,3968.0,Fantasy,Adventure,Animation,Family,NaN,NaN,NaN,NaN,8.033756
15480,27205,Inception,29.108149,Inception,8.1,14075.0,Action,Thriller,Science Fiction,Mystery,Adventure,NaN,NaN,NaN,8.025207


In [32]:
# Top Drama movies:
drama = filtered[(filtered[0] == 'Drama') |
         (filtered[1] == 'Drama') |
         (filtered[2] == 'Drama') |
         (filtered[3] == 'Drama') |
         (filtered[4] == 'Drama') |
         (filtered[5] == 'Drama') |
         (filtered[6] == 'Drama') |
         (filtered[7] == 'Drama')][['title','wr']]

In [33]:
drama[drama['wr'] < 7.835528]

,title,wr
24860,The Imitation Game,7.835528
359,The Lion King,7.825177
49,The Usual Suspects,7.812250
22841,The Grand Budapest Hotel,7.795044
11354,The Prestige,7.789494
32144,Room,7.768688
1201,Dead Poets Society,7.763345
22131,The Wolf of Wall Street,7.761535
30315,Inside Out,7.760937
1161,12 Angry Men,7.760290


In [37]:
movies.to_csv('./data/movies.csv')